# Wavenet Classifier 중간평가
* 작성자: 최정혁
* E-mail: [isnbh0@gmail.com](mailto:isnbh0@gmail.com)
* Github: [isnbh0](https://github.com/isnbh0)

## 목적
오디오로부터 특정한 feature를 먼저 추출해야지만 트레이닝이 가능한 기존 방법과 달리 순수 waveform만을 이용하여 소리를 분류한다.  
주 방법은 생성모형(generative model)으로 알려진 WaveNet의 구조를 개조하여 적용해 보는 것으로 한다.

### 기본구조: Raw Audio 처리가 가능한 Modified WaveNet
Dilated causal convolution을 이용하여 레이어 갯수 대비 receptive field 크기를 기하급수적으로 증가시킬 수 있도록 만든 CNN 구조. 0.01초의 소리에도 수백 개의 샘플이 들어가는 특징을 가진 오디오 데이터 분석이 가능함.
* Dilated: 인식 영역을 넓히기 위해 stride를 넓힌 형태의 convolution
* Causal: 소리는 기본적으로 시간성을 지녔으므로 특정 시간의 샘플에는 그 시간 이전 샘플의 정보만 들어있다고 할 수 있음
![Dilated causal convolution](https://raw.githubusercontent.com/isnbh0/wavenet-classifier/onoff_ver/_images/dcc.png)

#### 기존 아키텍처
* $n$개의 샘플이 주어졌을 때 $n+1$번째 샘플을 예측하기 위한 generative model 구조.
* 나올 수 있는 샘플의 값이 256단계로 양자화되어 softmax activation을 통해 256가지 중 하나의 값으로 분류/예측
* 구현 예시: [ibab@github](https://github.com/ibab/tensorflow-wavenet) (TensorFlow의 low-level (core) 기능들로 구현)
![Entire architecture](https://raw.githubusercontent.com/isnbh0/wavenet-classifier/onoff_ver/_images/entire.png)

#### 수정된 아키텍처
* __[현재 모델](https://github.com/isnbh0/wavenet-classifier/tree/onoff_ver/gcp_model/trainer)__ (ibab 모델을 수정)
* Binary classification을 위해 fully connected layer와 sigmoid activation을 거쳐 0과 1 사이 확률을 나타낼 스칼라값을 출력하도록 수정
* Loss function으로 binary cross-entropy 사용
$$\text{loss} = -\frac{1}{N}\sum_{i=1}^{N} \left( y_i \log{\hat{y_i}} + (1 - y_i) \log{(1 - \hat{y_i})} \right)$$
  * 이 경우 Benchmark (0.5 확률로 모두 찍는 경우의 loss): $-\log(0.5) \approx 0.693$
  * Loss 값 최소화가 목표
![Modified structure](https://raw.githubusercontent.com/isnbh0/wavenet-classifier/onoff_ver/_images/entire_mod.png)

#### 모델 구성에 사용되는 기본적인 파라미터
* `filter_width`: Dilated causal convolution (dilation) 단계에서 사용할 filter의 길이 (기본 `2`)
* `sample_rate`: 1초당 저장된 오디오 샘플 갯수 (기본 `22050`; `16000` 사용 가능)
* `dilations`: Dilation 레이어를 겹칠 때 순차적으로 쓰이는 dilation factor (기본 20 layers: `[1, 2, 4, 8, 16, 32, 64, 128, 256, 512]*2`)
* `residual_channels`: Residual 연산이 출력하는 Tensor의 채널 수 (기본 `16`)
* `dilation_channels`: Dilation layer가 출력하는 Tensor의 채널 수 (기본 `16`)
* `skip_channels`: Output으로 바로 연결되는 skip connection이 출력하는 Tensor의 채널 수 (기본 `16`)
* `quantization_channels`: 오디오 샘플을 $\mu$-law companding으로 양자화 및 one-hot-encoding 시 나눌 단계 수 (기본 `256`)
* `use_biases`: Bias 상수 사용 (기본 `True`)
* `scalar_input`: 오디오 샘플을 one-hot-encoding하지 않고 숫자로 받는지 여부 (기본 `False`)
* `initial_filter_width`: Scalar로 받을 시 causal convolution 단계에서 사용할 filter의 길이 (기본 `32`)

In [1]:
import json

with open('../model/wavenet_params.json', 'r') as f:
    wavenet_params = json.load(f)

print(wavenet_params)

{'filter_width': 2, 'sample_rate': 22050, 'dilations': [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512], 'residual_channels': 16, 'dilation_channels': 16, 'quantization_channels': 256, 'skip_channels': 16, 'use_biases': True, 'scalar_input': False, 'initial_filter_width': 32}


### 사용 데이터
TensorFlow에서 제공하는 Speech Commands Data Set v0.01에서 인용
* Sample rate 16kHz의 1초짜리 목소리 파일 4240개 (파일당 16000샘플)
  * "on"이라고 말하는 목소리 파일 2101개
  * "off"라고 말하는 목소리 파일 2139개
* Train용, Evaluation용, Test용 데이터로 나눠 TFRecord 포맷으로 저장
  * Train용 2480개
  * Evaluation용 880개
  * Test용 880개

### Training 시도
* Google Cloud Platform 이용해 트레이닝
* Batch size 40 (1 epoch = 62 steps)
* 50 epochs (3110 steps)

#### 결과
* 10 step마다 검증 시행 (총 311회)
* 누적 정확도 51.0% (Benchmark: 전부 다 off라고 했을 경우 정확도 50.5%)
* Evaluation loss는 시간이 지날수록 증가함 (overfitting 의심)

#### 퍼포먼스 향상 방법
* 단기간의 정확도 향상을 위해서는 기존 방법론인 MFCC(Mel frequency cepstral coefficients) 변환을 이용, 오디오 파일을 이미지화한 feature를 input으로 한 CNN 구조 활용
* 결과 도출 직전의 dense 레이어 추가
* 모델에서 변경 가능한 파라미터를 변경해가며 실험

#### Overfitting 대책
* Training loss는 꾸준히 감소하는데 evaluation loss가 발산한다는 것은 모델 어딘가에서 overfitting이 일어나고 있다는 의미
* 대책
  * Overfitting 방지 방법 사용
    * L2-regularization 활용
    * Dropout layer 추가

![Train loss](https://raw.githubusercontent.com/isnbh0/wavenet-classifier/onoff_ver/_images/025_trainloss.png)

![Eval loss](https://raw.githubusercontent.com/isnbh0/wavenet-classifier/onoff_ver/_images/025_evalloss.png)

---

#### 기타 특이점
* Tensorboard 확인 결과, 각 layer마다 bias 값들은 갱신되는데 weight 값들은 거의 변하지 않는 현상 발견
  * Learning rate를 바꿔가며 확인 결과 update 시에 weight 갱신은 정상적으로 작동하고 있지만, 그 변화의 정도가 굉장히 작은 듯 보임

![Dilation layer 12 biases](https://raw.githubusercontent.com/isnbh0/wavenet-classifier/onoff_ver/_images/layer12_biases.png)

![Dilation layer 12 weights](https://raw.githubusercontent.com/isnbh0/wavenet-classifier/onoff_ver/_images/layer12_weights.png)

### 이후 계획
* Regularization 기능 및 dropout layer 추가 시도
  * Google Cloud ML 엔진을 활용하여 L2 regularization의 hyperparameter 튜닝 시도
* Google Cloud Platform 상에서 distributed computing 및 GPU 활용한 트레이닝이 가능하도록 코드 개조
* Raw waveform 형태의 입력을 받는 기타 end-to-end audio classification 방법 리서치
* MFCC 변환 후 CNN 구조 이용하는 기존 방법론 구현